In [1]:
import numpy as np
import pandas as pd
import torch

from utlis import train_test_split_data
from datasets import Dataset

from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    set_seed,
)


KeyboardInterrupt: 

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [23]:
SEED = 42
set_seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device.type == "cuda":
    torch.cuda.manual_seed_all(SEED)

In [24]:
df = pd.read_pickle("../data/combined_df.pkl")

df["content_text_joined"] = df["content_words"].apply(lambda x: " ".join(x))

TEXT_COL = "content_text_joined"
LABEL_COL = "is_spoiler"

In [25]:
X_train, X_test, y_train, y_test = train_test_split_data(
    df,
    TEXT_COL,
    LABEL_COL
)

In [26]:
train_ds = Dataset.from_dict({
    "text": X_train,
    "labels": y_train.astype(int)
})

test_ds = Dataset.from_dict({
    "text": X_test,
    "labels": y_test.astype(int)
})

dataset = {
    "train": train_ds,
    "test": test_ds
}


In [27]:
MODEL_CHECKPOINT = "roberta-base"
NUM_LABELS = 2

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CHECKPOINT,
    num_labels=NUM_LABELS
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [29]:
def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=512
    )

tokenized_ds = {
    split: dataset[split].map(
        tokenize,
        batched=True,
        remove_columns=["text"]
    )
    for split in dataset
}

data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [30]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "mcc": matthews_corrcoef(labels, preds),
    }

In [31]:
training_args = TrainingArguments(
    output_dir="./results/RoBERTa",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="mcc",
    greater_is_better=True,
    logging_dir="./logs/RoBERTa",
    report_to="none",
    seed=SEED,
)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

C:\Users\User\AppData\Local\Temp\ipykernel_18360\4035158853.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [33]:
trainer.train()


  0%|          | 0/17500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
eval_results = trainer.evaluate()
print("Final evaluation:", eval_results)

In [ ]:
trainer.save_model("../Models/RoBERTa/final_model")
tokenizer.save_pretrained("../Models/RoBERTa/tokenizer")